In [6]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from snowflake.connector.pandas_tools import write_pandas
from sqlalchemy import create_engine
import snowflake.connector

In [7]:
import snowflake.connector

In [8]:
## Set options to view all columns
pd.set_option('display.max_columns', None)

In [9]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'TABLES'

In [10]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

In [11]:
cursor = conn.cursor()

In [12]:
cursor.execute('SELECT * FROM TEAMS')
team_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_names = pd.DataFrame(team_rows, columns=column_names)

In [13]:
cursor.execute('SELECT * FROM TEAM_ATTACKING_STATS')
attacking_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_attacking = pd.DataFrame(attacking_rows, columns=column_names)

In [14]:
cursor.execute('SELECT * FROM TEAM_DEFENDING_STATS')
defending_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_defending = pd.DataFrame(defending_rows, columns=column_names)

In [15]:
cursor.execute('SELECT * FROM TEAM_STANDARD_STATS')
standard_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_standard = pd.DataFrame(standard_rows, columns=column_names)

In [17]:
team_names.head()

,TEAM_NAME,TEAM_FBREF_ID
0,Arsenal,18bb7c10
1,Aston Villa,8602292d
2,Bournemouth,4ba7cbea
3,Brentford,cd051869
4,Brighton,d07537b9


In [20]:
df_table = team_standard[['TEAM_FBREF_ID', 'SEASON', 'TEAM_PTS', 'TEAM_XPTS']]

In [22]:
df_table = df_table.merge(team_names, how='left', on='TEAM_FBREF_ID')

In [27]:
df_table = df_table.merge(team_attacking[['TEAM_FBREF_ID', 'SEASON', 'GOALS_SCORED', 'XG']], on=['TEAM_FBREF_ID', 'SEASON'])

In [31]:
df_table = df_table.merge(team_defending[['TEAM_FBREF_ID', 'SEASON', 'GOALS_CONCEDED', 'XG_AGAINST']], on=['TEAM_FBREF_ID', 'SEASON'])

In [33]:
team_logos = \
[["18bb7c10", "https://i.imgur.com/SURo5sj.png"],
 ["8602292d", "https://i.imgur.com/M4mwH1X.png"],
 ["4ba7cbea", "https://i.imgur.com/0F6UIO4.png"],
 ["cd051869", "https://i.imgur.com/MOhGOQ4.png"],
 ["d07537b9", "https://i.imgur.com/EMCwD3X.png"],
 ["cff3d9bb", "https://i.imgur.com/6F7h3UR.png"],
 ["47c64c55", "https://i.imgur.com/LPhzr0K.png"],
 ["d3fd31cc", "https://i.imgur.com/Dgei0uj.png"],
 ["fd962109", "https://i.imgur.com/2zggSrF.png"],
 ["5bfb9659", "https://i.imgur.com/2tipH85.png"],
 ["a2d435b3", "https://i.imgur.com/ldOUO84.png"],
 ["822bd0ba", "https://i.imgur.com/fs7VK9G.png"],
 ["b8fd03ef", "https://i.imgur.com/FF8z7uZ.png"],
 ["19538871", "https://i.imgur.com/z47emBQ.png"],
 ["b2b47a98", "https://i.imgur.com/3kOLokZ.png"],
 ["e4a775cb", "https://i.imgur.com/z4Tl9Zu.png"],
 ["33c895d4", "https://i.imgur.com/35GrgJC.png"],
 ["361ca564", "https://i.imgur.com/fJ7rUWW.png"],
 ["7c21e445", "https://i.imgur.com/9BQHJRl.png"],
 ["8cec06e1", "https://i.imgur.com/6azSF88.png"]
 ]

In [32]:
df_table

,TEAM_FBREF_ID,SEASON,TEAM_PTS,TEAM_XPTS,TEAM_NAME,GOALS_SCORED,XG,GOALS_CONCEDED,XG_AGAINST
0,18bb7c10,2223,84,72.53,Arsenal,88,76.51,43,45.16
1,8602292d,2223,61,54.37,Aston Villa,51,50.00,46,53.96
2,4ba7cbea,2223,39,34.73,Bournemouth,37,40.08,71,67.60
3,cd051869,2223,59,58.26,Brentford,58,59.23,46,51.45
4,d07537b9,2223,62,70.86,Brighton,72,77.72,53,50.15
5,cff3d9bb,2223,44,52.28,Chelsea,38,51.89,47,54.02
6,47c64c55,2223,45,44.24,Crystal Palace,40,40.72,49,51.63
7,d3fd31cc,2223,36,40.71,Everton,34,48.65,57,68.58
8,fd962109,2223,52,39.24,Fulham,55,47.98,53,71.49
9,5bfb9659,2223,31,40.58,Leeds United,48,47.80,78,69.58
